In [8]:
!pip install rouge-score

In [9]:
from google.colab import files
from google.colab import drive
from google.colab import userdata
import nltk
import pandas as pd
import csv
from rouge_score import rouge_scorer

In [10]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
output_file = "/content/drive/My Drive/sum_test.csv"

df = pd.read_csv(output_file)
df.head()

,bug_report,ground_truth_summary
0,Build ID: 20090619-0625 Steps To Reproduce: 0...,[ui] IU Properties dialog improvements
1,The Equinox Resources page (see url) has outda...,Outdated links on website
2,When the hierarchy view has no corresponding s...,"TVT: The ""empty"" Hierarchy view message is con..."
3,"In the 1.0 version of Eclipse, when you place ...",Bracket matching should highlight both brackets
4,Build 20021216 The Java->JUnit preference pag...,[JUnit] Preference page: stack filter should b...


In [12]:
df['bug.id'] = df.index
df.head()

,bug_report,ground_truth_summary,bug.id
0,Build ID: 20090619-0625 Steps To Reproduce: 0...,[ui] IU Properties dialog improvements,0
1,The Equinox Resources page (see url) has outda...,Outdated links on website,1
2,When the hierarchy view has no corresponding s...,"TVT: The ""empty"" Hierarchy view message is con...",2
3,"In the 1.0 version of Eclipse, when you place ...",Bracket matching should highlight both brackets,3
4,Build 20021216 The Java->JUnit preference pag...,[JUnit] Preference page: stack filter should b...,4


#CodeLlama

In [ ]:
output_file2 = "/content/drive/My Drive/codellama-br-ft-eval-sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,Bad URL in the install theme button
1,26540,The dropdown menu for Device Pixel Ratio (DPR)...
2,4320,1. Compiler optimizes out a local variable tha...
3,14545,"Password Manager Disabled, No Popup Appears"
4,23198,irc connection is not being established.


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,Bad URL in the install theme button,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,The dropdown menu for Device Pixel Ratio (DPR)...,[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,1. Compiler optimizes out a local variable tha...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,"Password Manager Disabled, No Popup Appears",Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,irc connection is not being established.,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from rouge_score import rouge_scorer

def evaluate_rouge(ground_truth_summary, summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ground_truth_summary, summary)
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in valid_rows.index:
    gt = df2.at[idx, "ground_truth_summary"]
    gen = df2.at[idx, "summary"]
    r1, r2, rl = evaluate_rouge(gt, gen)
    rouge1_scores.append(r1)
    rouge2_scores.append(r2)
    rougeL_scores.append(rl)
    print(f"Row {idx}, Bug ID {df2.at[idx, 'bug.id']}: "
          f"ROUGE-1={r1:.4f}, ROUGE-2={r2:.4f}, ROUGE-L={rl:.4f}")

df2.loc[valid_rows.index, "rouge1"] = rouge1_scores
df2.loc[valid_rows.index, "rouge2"] = rouge2_scores
df2.loc[valid_rows.index, "rougeL"] = rougeL_scores

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print("\n=== Average ROUGE Scores ===")
print(f"ROUGE-1: {avg_rouge1:.4f}, ROUGE-2: {avg_rouge2:.4f}, ROUGE-L: {avg_rougeL:.4f}")


Row 0, Bug ID 18027: ROUGE-1=0.4000, ROUGE-2=0.1538, ROUGE-L=0.2667
Row 1, Bug ID 26540: ROUGE-1=0.1714, ROUGE-2=0.0000, ROUGE-L=0.1143
Row 2, Bug ID 4320: ROUGE-1=0.3043, ROUGE-2=0.0455, ROUGE-L=0.2174
Row 3, Bug ID 14545: ROUGE-1=0.3810, ROUGE-2=0.1053, ROUGE-L=0.2857
Row 4, Bug ID 23198: ROUGE-1=0.1667, ROUGE-2=0.0000, ROUGE-L=0.1667
Row 5, Bug ID 5663: ROUGE-1=0.1053, ROUGE-2=0.0000, ROUGE-L=0.1053
Row 6, Bug ID 9750: ROUGE-1=0.5455, ROUGE-2=0.2222, ROUGE-L=0.5455
Row 7, Bug ID 18076: ROUGE-1=0.2000, ROUGE-2=0.0000, ROUGE-L=0.2000
Row 8, Bug ID 6580: ROUGE-1=0.0952, ROUGE-2=0.0000, ROUGE-L=0.0952
Row 9, Bug ID 4252: ROUGE-1=0.1875, ROUGE-2=0.0000, ROUGE-L=0.1250
Row 10, Bug ID 12667: ROUGE-1=0.9091, ROUGE-2=0.6667, ROUGE-L=0.9091
Row 11, Bug ID 2805: ROUGE-1=0.0741, ROUGE-2=0.0000, ROUGE-L=0.0741
Row 12, Bug ID 5339: ROUGE-1=0.0000, ROUGE-2=0.0000, ROUGE-L=0.0000
Row 13, Bug ID 6462: ROUGE-1=0.1053, ROUGE-2=0.0000, ROUGE-L=0.1053
Row 14, Bug ID 17436: ROUGE-1=0.4000, ROUGE-2=0.3077

#Llama3.1

In [ ]:
output_file2 = "/content/drive/My Drive/llama3-br-ft-eval-sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,Mozilla crashes when installing a theme with a...
1,26540,The DPR dropdown and actions are not visible o...
2,4320,Optimizer incorrectly removes unused local var...
3,14545,The insecure password warning does not appear ...
4,23198,The connection to the IRC server is not being ...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,Mozilla crashes when installing a theme with a...,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,The DPR dropdown and actions are not visible o...,[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,Optimizer incorrectly removes unused local var...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,The insecure password warning does not appear ...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,The connection to the IRC server is not being ...,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from rouge_score import rouge_scorer

def evaluate_rouge(ground_truth_summary, summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ground_truth_summary, summary)
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in valid_rows.index:
    gt = df2.at[idx, "ground_truth_summary"]
    gen = df2.at[idx, "summary"]
    r1, r2, rl = evaluate_rouge(gt, gen)
    rouge1_scores.append(r1)
    rouge2_scores.append(r2)
    rougeL_scores.append(rl)
    print(f"Row {idx}, Bug ID {df2.at[idx, 'bug.id']}: "
          f"ROUGE-1={r1:.4f}, ROUGE-2={r2:.4f}, ROUGE-L={rl:.4f}")

df2.loc[valid_rows.index, "rouge1"] = rouge1_scores
df2.loc[valid_rows.index, "rouge2"] = rouge2_scores
df2.loc[valid_rows.index, "rougeL"] = rougeL_scores

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print("\n=== Average ROUGE Scores ===")
print(f"ROUGE-1: {avg_rouge1:.4f}, ROUGE-2: {avg_rouge2:.4f}, ROUGE-L: {avg_rougeL:.4f}")


Row 0, Bug ID 18027: ROUGE-1=0.1205, ROUGE-2=0.0494, ROUGE-L=0.1205
Row 1, Bug ID 26540: ROUGE-1=0.1905, ROUGE-2=0.0194, ROUGE-L=0.1333
Row 2, Bug ID 4320: ROUGE-1=0.1758, ROUGE-2=0.0449, ROUGE-L=0.1099
Row 3, Bug ID 14545: ROUGE-1=0.2062, ROUGE-2=0.0632, ROUGE-L=0.1649
Row 4, Bug ID 23198: ROUGE-1=0.0674, ROUGE-2=0.0460, ROUGE-L=0.0674
Row 5, Bug ID 5663: ROUGE-1=0.2051, ROUGE-2=0.0526, ROUGE-L=0.1282
Row 6, Bug ID 9750: ROUGE-1=0.0870, ROUGE-2=0.0299, ROUGE-L=0.0870
Row 7, Bug ID 18076: ROUGE-1=0.1111, ROUGE-2=0.0000, ROUGE-L=0.0667
Row 8, Bug ID 6580: ROUGE-1=0.1067, ROUGE-2=0.0000, ROUGE-L=0.1067
Row 9, Bug ID 4252: ROUGE-1=0.0444, ROUGE-2=0.0000, ROUGE-L=0.0444
Row 10, Bug ID 12667: ROUGE-1=0.1333, ROUGE-2=0.0548, ROUGE-L=0.1067
Row 11, Bug ID 2805: ROUGE-1=0.1034, ROUGE-2=0.0000, ROUGE-L=0.1034
Row 12, Bug ID 5339: ROUGE-1=0.0606, ROUGE-2=0.0000, ROUGE-L=0.0606
Row 13, Bug ID 6462: ROUGE-1=0.0541, ROUGE-2=0.0000, ROUGE-L=0.0541
Row 14, Bug ID 17436: ROUGE-1=0.1538, ROUGE-2=0.0541

#Mistral

In [ ]:
output_file2 = "/content/drive/My Drive/mistral-br-ft-eval-sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,Theme installer crashes with malformed URL input.
1,26540,Dark theme hides device selection drop-down in...
2,4320,Optimizing unused local variables removes fiel...
3,14545,Password manager warning does not show for in ...
4,23198,Connection to IRC server keeps getting closed.


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,Theme installer crashes with malformed URL input.,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,Dark theme hides device selection drop-down in...,[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,Optimizing unused local variables removes fiel...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,Password manager warning does not show for in ...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,Connection to IRC server keeps getting closed.,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from rouge_score import rouge_scorer

def evaluate_rouge(ground_truth_summary, summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ground_truth_summary, summary)
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in valid_rows.index:
    gt = df2.at[idx, "ground_truth_summary"]
    gen = df2.at[idx, "summary"]
    r1, r2, rl = evaluate_rouge(gt, gen)
    rouge1_scores.append(r1)
    rouge2_scores.append(r2)
    rougeL_scores.append(rl)
    print(f"Row {idx}, Bug ID {df2.at[idx, 'bug.id']}: "
          f"ROUGE-1={r1:.4f}, ROUGE-2={r2:.4f}, ROUGE-L={rl:.4f}")

df2.loc[valid_rows.index, "rouge1"] = rouge1_scores
df2.loc[valid_rows.index, "rouge2"] = rouge2_scores
df2.loc[valid_rows.index, "rougeL"] = rougeL_scores

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print("\n=== Average ROUGE Scores ===")
print(f"ROUGE-1: {avg_rouge1:.4f}, ROUGE-2: {avg_rouge2:.4f}, ROUGE-L: {avg_rougeL:.4f}")


Row 0, Bug ID 18027: ROUGE-1=0.2667, ROUGE-2=0.0000, ROUGE-L=0.2667
Row 1, Bug ID 26540: ROUGE-1=0.3704, ROUGE-2=0.0800, ROUGE-L=0.1481
Row 2, Bug ID 4320: ROUGE-1=0.2857, ROUGE-2=0.1053, ROUGE-L=0.1905
Row 3, Bug ID 14545: ROUGE-1=0.3333, ROUGE-2=0.0714, ROUGE-L=0.2667
Row 4, Bug ID 23198: ROUGE-1=0.1538, ROUGE-2=0.0000, ROUGE-L=0.1538
Row 5, Bug ID 5663: ROUGE-1=0.4000, ROUGE-2=0.0870, ROUGE-L=0.4000
Row 6, Bug ID 9750: ROUGE-1=0.3636, ROUGE-2=0.0000, ROUGE-L=0.3636
Row 7, Bug ID 18076: ROUGE-1=0.2727, ROUGE-2=0.0000, ROUGE-L=0.1818
Row 8, Bug ID 6580: ROUGE-1=0.0417, ROUGE-2=0.0000, ROUGE-L=0.0417
Row 9, Bug ID 4252: ROUGE-1=0.4000, ROUGE-2=0.0000, ROUGE-L=0.2667
Row 10, Bug ID 12667: ROUGE-1=0.8333, ROUGE-2=0.6000, ROUGE-L=0.8333
Row 11, Bug ID 2805: ROUGE-1=0.4706, ROUGE-2=0.1333, ROUGE-L=0.4706
Row 12, Bug ID 5339: ROUGE-1=0.2222, ROUGE-2=0.0000, ROUGE-L=0.2222
Row 13, Bug ID 6462: ROUGE-1=0.3333, ROUGE-2=0.0000, ROUGE-L=0.2500
Row 14, Bug ID 17436: ROUGE-1=0.2857, ROUGE-2=0.2105

#Phi-3

In [ ]:
output_file2 = "/content/drive/My Drive/phi-3-br-ft-eval-sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,Bad URL in Install theme button causes Firefox...
1,26540,"Dark theme hides DPM/DPR indicators, misleadin..."
2,4320,Compiler optimizes away unreferenced local var...
3,14545,Incorrect fix causes password warning to not a...
4,23198,Connection to chat server closed unexpectedly....


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,Bad URL in Install theme button causes Firefox...,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,"Dark theme hides DPM/DPR indicators, misleadin...",[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,Compiler optimizes away unreferenced local var...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,Incorrect fix causes password warning to not a...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,Connection to chat server closed unexpectedly....,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from rouge_score import rouge_scorer

def evaluate_rouge(ground_truth_summary, summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ground_truth_summary, summary)
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in valid_rows.index:
    gt = df2.at[idx, "ground_truth_summary"]
    gen = df2.at[idx, "summary"]
    r1, r2, rl = evaluate_rouge(gt, gen)
    rouge1_scores.append(r1)
    rouge2_scores.append(r2)
    rougeL_scores.append(rl)
    print(f"Row {idx}, Bug ID {df2.at[idx, 'bug.id']}: "
          f"ROUGE-1={r1:.4f}, ROUGE-2={r2:.4f}, ROUGE-L={rl:.4f}")

df2.loc[valid_rows.index, "rouge1"] = rouge1_scores
df2.loc[valid_rows.index, "rouge2"] = rouge2_scores
df2.loc[valid_rows.index, "rougeL"] = rougeL_scores

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print("\n=== Average ROUGE Scores ===")
print(f"ROUGE-1: {avg_rouge1:.4f}, ROUGE-2: {avg_rouge2:.4f}, ROUGE-L: {avg_rougeL:.4f}")


Row 0, Bug ID 18027: ROUGE-1=0.1081, ROUGE-2=0.0278, ROUGE-L=0.0811
Row 1, Bug ID 26540: ROUGE-1=0.1951, ROUGE-2=0.0513, ROUGE-L=0.1951
Row 2, Bug ID 4320: ROUGE-1=0.1875, ROUGE-2=0.0000, ROUGE-L=0.1250
Row 3, Bug ID 14545: ROUGE-1=0.2118, ROUGE-2=0.0241, ROUGE-L=0.1647
Row 4, Bug ID 23198: ROUGE-1=0.0563, ROUGE-2=0.0000, ROUGE-L=0.0563
Row 5, Bug ID 5663: ROUGE-1=0.2022, ROUGE-2=0.0230, ROUGE-L=0.1124
Row 6, Bug ID 9750: ROUGE-1=0.0779, ROUGE-2=0.0267, ROUGE-L=0.0779
Row 7, Bug ID 18076: ROUGE-1=0.1702, ROUGE-2=0.0000, ROUGE-L=0.0851
Row 8, Bug ID 6580: ROUGE-1=0.1000, ROUGE-2=0.0000, ROUGE-L=0.0667
Row 9, Bug ID 4252: ROUGE-1=0.0833, ROUGE-2=0.0286, ROUGE-L=0.0833
Row 10, Bug ID 12667: ROUGE-1=0.2069, ROUGE-2=0.0741, ROUGE-L=0.2069
Row 11, Bug ID 2805: ROUGE-1=0.0882, ROUGE-2=0.0000, ROUGE-L=0.0882
Row 12, Bug ID 5339: ROUGE-1=0.0500, ROUGE-2=0.0000, ROUGE-L=0.0500
Row 13, Bug ID 6462: ROUGE-1=0.1818, ROUGE-2=0.0000, ROUGE-L=0.1818
Row 14, Bug ID 17436: ROUGE-1=0.0588, ROUGE-2=0.0303

#Gemma

In [ ]:
output_file2 = "/content/drive/My Drive/gemma-br-ft-eval-sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,"Sure, here is a summary in one sentence : The ..."
1,26540,"Sure, here is a summary in one sentence:\n\nTh..."
2,4320,"Sure, here's a summary in one sentence: The c..."
3,14545,Password manager disables insecure password wa...
4,23198,The connection to the server is being closed p...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,"Sure, here is a summary in one sentence : The ...",I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,"Sure, here is a summary in one sentence:\n\nTh...",[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,"Sure, here's a summary in one sentence: The c...","Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,Password manager disables insecure password wa...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,The connection to the server is being closed p...,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from rouge_score import rouge_scorer

def evaluate_rouge(ground_truth_summary, summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ground_truth_summary, summary)
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in valid_rows.index:
    gt = df2.at[idx, "ground_truth_summary"]
    gen = df2.at[idx, "summary"]
    r1, r2, rl = evaluate_rouge(gt, gen)
    rouge1_scores.append(r1)
    rouge2_scores.append(r2)
    rougeL_scores.append(rl)
    print(f"Row {idx}, Bug ID {df2.at[idx, 'bug.id']}: "
          f"ROUGE-1={r1:.4f}, ROUGE-2={r2:.4f}, ROUGE-L={rl:.4f}")

df2.loc[valid_rows.index, "rouge1"] = rouge1_scores
df2.loc[valid_rows.index, "rouge2"] = rouge2_scores
df2.loc[valid_rows.index, "rougeL"] = rougeL_scores

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print("\n=== Average ROUGE Scores ===")
print(f"ROUGE-1: {avg_rouge1:.4f}, ROUGE-2: {avg_rouge2:.4f}, ROUGE-L: {avg_rougeL:.4f}")


Row 0, Bug ID 18027: ROUGE-1=0.1860, ROUGE-2=0.0000, ROUGE-L=0.1860
Row 1, Bug ID 26540: ROUGE-1=0.2727, ROUGE-2=0.0476, ROUGE-L=0.1818
Row 2, Bug ID 4320: ROUGE-1=0.2667, ROUGE-2=0.0465, ROUGE-L=0.1778
Row 3, Bug ID 14545: ROUGE-1=0.5217, ROUGE-2=0.1905, ROUGE-L=0.2609
Row 4, Bug ID 23198: ROUGE-1=0.2667, ROUGE-2=0.1538, ROUGE-L=0.2667
Row 5, Bug ID 5663: ROUGE-1=0.4211, ROUGE-2=0.0556, ROUGE-L=0.3158
Row 6, Bug ID 9750: ROUGE-1=0.1667, ROUGE-2=0.0909, ROUGE-L=0.1667
Row 7, Bug ID 18076: ROUGE-1=0.2222, ROUGE-2=0.0000, ROUGE-L=0.1481
Row 8, Bug ID 6580: ROUGE-1=0.1905, ROUGE-2=0.0000, ROUGE-L=0.0476
Row 9, Bug ID 4252: ROUGE-1=0.1667, ROUGE-2=0.0000, ROUGE-L=0.1667
Row 10, Bug ID 12667: ROUGE-1=0.2222, ROUGE-2=0.0800, ROUGE-L=0.2222
Row 11, Bug ID 2805: ROUGE-1=0.1333, ROUGE-2=0.0000, ROUGE-L=0.1333
Row 12, Bug ID 5339: ROUGE-1=0.0625, ROUGE-2=0.0000, ROUGE-L=0.0625
Row 13, Bug ID 6462: ROUGE-1=0.1622, ROUGE-2=0.0000, ROUGE-L=0.1081
Row 14, Bug ID 17436: ROUGE-1=0.2759, ROUGE-2=0.1481

Qwen3

In [ ]:
output_file2 = "/content/drive/My Drive/qwen-br-ft-eval-sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,The user is unable to install a theme due to a...
1,26540,The issue is that the DPR dropdown and actions...
2,4320,The issue is that the compiler fails to recogn...
3,14545,The bug is that when the Password Manager is t...
4,23198,The user is unable to connect to the IRC serve...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,The user is unable to install a theme due to a...,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,The issue is that the DPR dropdown and actions...,[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,The issue is that the compiler fails to recogn...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,The bug is that when the Password Manager is t...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,The user is unable to connect to the IRC serve...,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from rouge_score import rouge_scorer

def evaluate_rouge(ground_truth_summary, summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ground_truth_summary, summary)
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in valid_rows.index:
    gt = df2.at[idx, "ground_truth_summary"]
    gen = df2.at[idx, "summary"]
    r1, r2, rl = evaluate_rouge(gt, gen)
    rouge1_scores.append(r1)
    rouge2_scores.append(r2)
    rougeL_scores.append(rl)
    print(f"Row {idx}, Bug ID {df2.at[idx, 'bug.id']}: "
          f"ROUGE-1={r1:.4f}, ROUGE-2={r2:.4f}, ROUGE-L={rl:.4f}")

df2.loc[valid_rows.index, "rouge1"] = rouge1_scores
df2.loc[valid_rows.index, "rouge2"] = rouge2_scores
df2.loc[valid_rows.index, "rougeL"] = rougeL_scores

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print("\n=== Average ROUGE Scores ===")
print(f"ROUGE-1: {avg_rouge1:.4f}, ROUGE-2: {avg_rouge2:.4f}, ROUGE-L: {avg_rougeL:.4f}")


Row 0, Bug ID 18027: ROUGE-1=0.0833, ROUGE-2=0.0000, ROUGE-L=0.0833
Row 1, Bug ID 26540: ROUGE-1=0.1667, ROUGE-2=0.0213, ROUGE-L=0.1250
Row 2, Bug ID 4320: ROUGE-1=0.1429, ROUGE-2=0.0000, ROUGE-L=0.1020
Row 3, Bug ID 14545: ROUGE-1=0.2200, ROUGE-2=0.0612, ROUGE-L=0.1200
Row 4, Bug ID 23198: ROUGE-1=0.0632, ROUGE-2=0.0430, ROUGE-L=0.0421
Row 5, Bug ID 5663: ROUGE-1=0.1765, ROUGE-2=0.0400, ROUGE-L=0.1373
Row 6, Bug ID 9750: ROUGE-1=0.0741, ROUGE-2=0.0253, ROUGE-L=0.0741
Row 7, Bug ID 18076: ROUGE-1=0.0619, ROUGE-2=0.0000, ROUGE-L=0.0619
Row 8, Bug ID 6580: ROUGE-1=0.0784, ROUGE-2=0.0000, ROUGE-L=0.0588
Row 9, Bug ID 4252: ROUGE-1=0.0667, ROUGE-2=0.0000, ROUGE-L=0.0444
Row 10, Bug ID 12667: ROUGE-1=0.1010, ROUGE-2=0.0412, ROUGE-L=0.1010
Row 11, Bug ID 2805: ROUGE-1=0.0825, ROUGE-2=0.0421, ROUGE-L=0.0825
Row 12, Bug ID 5339: ROUGE-1=0.0426, ROUGE-2=0.0000, ROUGE-L=0.0213
Row 13, Bug ID 6462: ROUGE-1=0.1087, ROUGE-2=0.0000, ROUGE-L=0.0652
Row 14, Bug ID 17436: ROUGE-1=0.0851, ROUGE-2=0.0217

#Deepseek Coder

In [ ]:
output_file2 = "/content/drive/My Drive/deepseek-br-ft-eval-sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,The bug occurs when trying to install a theme ...
1,26540,The DevEdition Dynamic Pixel Ratio (DPR) optio...
2,4320,The compiler does not generate a call to the c...
3,14545,Password Manager is not working as expected wh...
4,23198,Connection to IRC server is failing due to SOC...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,The bug occurs when trying to install a theme ...,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,The DevEdition Dynamic Pixel Ratio (DPR) optio...,[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,The compiler does not generate a call to the c...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,Password Manager is not working as expected wh...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,Connection to IRC server is failing due to SOC...,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from rouge_score import rouge_scorer

def evaluate_rouge(ground_truth_summary, summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ground_truth_summary, summary)
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in valid_rows.index:
    gt = df2.at[idx, "ground_truth_summary"]
    gen = df2.at[idx, "summary"]
    r1, r2, rl = evaluate_rouge(gt, gen)
    rouge1_scores.append(r1)
    rouge2_scores.append(r2)
    rougeL_scores.append(rl)
    print(f"Row {idx}, Bug ID {df2.at[idx, 'bug.id']}: "
          f"ROUGE-1={r1:.4f}, ROUGE-2={r2:.4f}, ROUGE-L={rl:.4f}")

df2.loc[valid_rows.index, "rouge1"] = rouge1_scores
df2.loc[valid_rows.index, "rouge2"] = rouge2_scores
df2.loc[valid_rows.index, "rougeL"] = rougeL_scores

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print("\n=== Average ROUGE Scores ===")
print(f"ROUGE-1: {avg_rouge1:.4f}, ROUGE-2: {avg_rouge2:.4f}, ROUGE-L: {avg_rougeL:.4f}")


Row 0, Bug ID 18027: ROUGE-1=0.1149, ROUGE-2=0.0471, ROUGE-L=0.1149
Row 1, Bug ID 26540: ROUGE-1=0.2045, ROUGE-2=0.0233, ROUGE-L=0.1136
Row 2, Bug ID 4320: ROUGE-1=0.1124, ROUGE-2=0.0000, ROUGE-L=0.0899
Row 3, Bug ID 14545: ROUGE-1=0.2000, ROUGE-2=0.0455, ROUGE-L=0.2000
Row 4, Bug ID 23198: ROUGE-1=0.0732, ROUGE-2=0.0000, ROUGE-L=0.0732
Row 5, Bug ID 5663: ROUGE-1=0.2069, ROUGE-2=0.0357, ROUGE-L=0.1724
Row 6, Bug ID 9750: ROUGE-1=0.3750, ROUGE-2=0.1429, ROUGE-L=0.3750
Row 7, Bug ID 18076: ROUGE-1=0.1087, ROUGE-2=0.0222, ROUGE-L=0.0652
Row 8, Bug ID 6580: ROUGE-1=0.1071, ROUGE-2=0.0000, ROUGE-L=0.0714
Row 9, Bug ID 4252: ROUGE-1=0.0870, ROUGE-2=0.0000, ROUGE-L=0.0870
Row 10, Bug ID 12667: ROUGE-1=0.1333, ROUGE-2=0.0465, ROUGE-L=0.1333
Row 11, Bug ID 2805: ROUGE-1=0.0638, ROUGE-2=0.0217, ROUGE-L=0.0638
Row 12, Bug ID 5339: ROUGE-1=0.0632, ROUGE-2=0.0000, ROUGE-L=0.0632
Row 13, Bug ID 6462: ROUGE-1=0.0488, ROUGE-2=0.0000, ROUGE-L=0.0488
Row 14, Bug ID 17436: ROUGE-1=0.0460, ROUGE-2=0.0235

#GPT 3.5-Turbo

In [13]:
output_file2 = "/content/drive/My Drive/gpt3.5_eval_sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,Mozilla blocks and crashes when using invalid ...
1,26540,Dark theme makes DPR difficult to see/click in...
2,4320,Optimization bug in SingleNameReference affect...
3,14545,Insecure password warning not showing with dis...
4,23198,Connection errors with IRC server using proxy.


In [14]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,Mozilla blocks and crashes when using invalid ...,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,Dark theme makes DPR difficult to see/click in...,[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,Optimization bug in SingleNameReference affect...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,Insecure password warning not showing with dis...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,Connection errors with IRC server using proxy.,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [15]:
from rouge_score import rouge_scorer

def evaluate_rouge(ground_truth_summary, summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ground_truth_summary, summary)
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in valid_rows.index:
    gt = df2.at[idx, "ground_truth_summary"]
    gen = df2.at[idx, "summary"]
    r1, r2, rl = evaluate_rouge(gt, gen)
    rouge1_scores.append(r1)
    rouge2_scores.append(r2)
    rougeL_scores.append(rl)
    print(f"Row {idx}, Bug ID {df2.at[idx, 'bug.id']}: "
          f"ROUGE-1={r1:.4f}, ROUGE-2={r2:.4f}, ROUGE-L={rl:.4f}")

df2.loc[valid_rows.index, "rouge1"] = rouge1_scores
df2.loc[valid_rows.index, "rouge2"] = rouge2_scores
df2.loc[valid_rows.index, "rougeL"] = rougeL_scores

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print("\n=== Average ROUGE Scores ===")
print(f"ROUGE-1: {avg_rouge1:.4f}, ROUGE-2: {avg_rouge2:.4f}, ROUGE-L: {avg_rougeL:.4f}")


Row 0, Bug ID 18027: ROUGE-1=0.3158, ROUGE-2=0.0000, ROUGE-L=0.1053
Row 1, Bug ID 26540: ROUGE-1=0.2963, ROUGE-2=0.0800, ROUGE-L=0.1481
Row 2, Bug ID 4320: ROUGE-1=0.0833, ROUGE-2=0.0000, ROUGE-L=0.0833
Row 3, Bug ID 14545: ROUGE-1=0.5000, ROUGE-2=0.1818, ROUGE-L=0.4167
Row 4, Bug ID 23198: ROUGE-1=0.1538, ROUGE-2=0.0000, ROUGE-L=0.1538
Row 5, Bug ID 5663: ROUGE-1=0.3636, ROUGE-2=0.1000, ROUGE-L=0.3636
Row 6, Bug ID 9750: ROUGE-1=0.4615, ROUGE-2=0.0000, ROUGE-L=0.4615
Row 7, Bug ID 18076: ROUGE-1=0.1905, ROUGE-2=0.0000, ROUGE-L=0.1905
Row 8, Bug ID 6580: ROUGE-1=0.0870, ROUGE-2=0.0000, ROUGE-L=0.0870
Row 9, Bug ID 4252: ROUGE-1=0.3158, ROUGE-2=0.1176, ROUGE-L=0.2105
Row 10, Bug ID 12667: ROUGE-1=0.6667, ROUGE-2=0.4615, ROUGE-L=0.6667
Row 11, Bug ID 2805: ROUGE-1=0.1333, ROUGE-2=0.0000, ROUGE-L=0.1333
Row 12, Bug ID 5339: ROUGE-1=0.0000, ROUGE-2=0.0000, ROUGE-L=0.0000
Row 13, Bug ID 6462: ROUGE-1=0.3000, ROUGE-2=0.0000, ROUGE-L=0.3000
Row 14, Bug ID 17436: ROUGE-1=0.5000, ROUGE-2=0.1429